In [ ]:
%pip uninstall hmmlearn

In [12]:
!pip install hmmlearn==0.2.6

     ------------------------------------ 118.7/118.7 kB 986.4 kB/s eta 0:00:00
  Attempting uninstall: hmmlearn
    Found existing installation: hmmlearn 0.2.8
    Uninstalling hmmlearn-0.2.8:
      Successfully uninstalled hmmlearn-0.2.8


ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Users\\fatib\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages\\~mmlearn\\_hmmc.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [13]:
import math
import numpy as np
from hmmlearn import hmm

states = ('Rainy', 'Sunny')
observations = ('walk', 'shop', 'clean')
 
start_probability = {'Rainy': 0.6, 'Sunny': 0.4}
 
transition_probability = {
   'Rainy' : {'Rainy': 0.7, 'Sunny': 0.3},
   'Sunny' : {'Rainy': 0.4, 'Sunny': 0.6},
   }
 
emission_probability = {
   'Rainy' : {'walk': 0.1, 'shop': 0.4, 'clean': 0.5},
   'Sunny' : {'walk': 0.6, 'shop': 0.3, 'clean': 0.1},
   }

model = hmm.MultinomialHMM(n_components=2) #number of hidden states
model.startprob_ = np.array([0.6, 0.4])
model.transmat_ = np.array([[0.7, 0.3],
                            [0.4, 0.6]])
model.emissionprob_ = np.array([[0.1, 0.4, 0.5],
                                [0.6, 0.3, 0.1]])

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Problem 1: Given a known model what is the likelihood of sequence O happening?

Answer: The probability of the first observation being “walk” equals to the multiplication of the initial state distribution and emission probability matrix. 0.6 x 0.1 + 0.4 x 0.6 = 0.30 (30%). The log likelihood is provided from calling .score.

In [14]:
walk = 0; shop = 1; clean = 2

p1 = math.exp(model.score(np.array([[walk]])))

p2 = math.exp(model.score(np.array([[shop]])))

p3 = math.exp(model.score(np.array([[clean]])))

p4 = math.exp(model.score(np.array([[clean,clean,clean]])))

print(p1,p2,p3,p4)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

Problem 2: Given a known model and sequence O, what is the optimal hidden state sequence? This will be useful if we want to know if the weather is “Rainy” or “Sunny”.

In [ ]:
logprob, seq = model.decode(np.array([[shop,clean,walk]]).transpose())
print(math.exp(logprob))
print(['Rainy' if x==0 else 'Sunny' for x in seq])

logprob, seq = model.decode(np.array([[clean,clean,clean]]).transpose())
print(math.exp(logprob))
print(['Rainy' if x==0 else 'Sunny' for x in seq])

0.015120000000000003
['Rainy', 'Rainy', 'Sunny']
0.03674999999999999
['Rainy', 'Rainy', 'Rainy']


- Given the known model and the observation {“Shop”, “Clean”, “Walk”}, the weather was most likely {“Rainy”, “Rainy”, “Sunny”} with ~1.5% probability.
- Given the known model and the observation {“Clean”, “Clean”, “Clean”}, the weather was most likely {“Rainy”, “Rainy”, “Rainy”} with ~3.6% probability.